In [ ]:
# umbridge server
from scipy.integrate import solve_ivp # for solving the ode
import nest_asyncio
nest_asyncio.apply()
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.special as special
from mpl_toolkits.mplot3d import axes3d
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
from scipy.linalg import block_diag
import umbridge
from scipy import linalg

class PDEModel(umbridge.Model):
    def __init__(self):
        super().__init__("forward")
        self.C= np.diag([2, 0, 0, 0, 2, 2])
        self.T = np.ones((6, 6))
        self.T[0] = 1/np.sqrt(6) * np.ones((1,6))
        self.T[1, 0:3] = - 1/np.sqrt(6) * np.ones((1,3))
        self.T[1, 3:6] = 1/np.sqrt(6) * np.ones((1,3))
        self.T[2] = [1/2, -1/2, 0, -1/2, 1/2, 0]
        self.T[3] = [1/ np.sqrt(12), 1/ np.sqrt(12), -1/ np.sqrt(3), -1/ np.sqrt(12), -1/ np.sqrt(12), 1/ np.sqrt(3) ]
        self.T[4] = [-1/2, 1/2, 0, -1/2, 1/2, 0]
        self.T[5] = [-1/ np.sqrt(12), -1/ np.sqrt(12), 1/ np.sqrt(3), -1/ np.sqrt(12), -1/ np.sqrt(12), 1/ np.sqrt(3) ]
        self.n = 6
        self.external_noise = [0] * 4  # Initialize with the required size (4 elements)
        self.u = [0]*6   # Initialize with the required size (6 elements)
        self.U_AC = 325
        self.U_DC = 700

        
        
        
    def get_input_sizes(self, config):
        return [1, 1, 1, 1, 1, 1, 42, 6, 1, 161] 
 # for the values 6 is for R_arm, L_arm etc., 20 is for p values, initial conditions for x, angular frequency, time  
    
    def get_output_sizes(self, config):
        return [161, 161, 161, 161, 161, 161] # for the output variables x_1:6
        
    def __call__(self, parameters, config):
        R_arm = parameters [0][0]
        R_DC = parameters [1][0]
        R_AC = parameters [2][0]
        L_arm = parameters [3][0]
        L_DC = parameters [4][0]
        L_AC = parameters [5][0]
        self.p = parameters[6]
        x_initial = parameters [7]
        self.omega = parameters[8][0]
        T = parameters[9]
        matrix_A = self.a(self.n, R_arm, R_DC, L_arm, L_DC, R_AC, L_AC, config)
        matrix_B = self.b(self.n, L_arm, L_DC, L_AC, config)
        matrix_F = self.f(self.n, L_arm, L_DC, L_AC, config)
        sol = solve_ivp(self.MMC,  [T[0],T[160]], x_initial, args=(matrix_A, matrix_B, matrix_F, config), method='RK45', t_eval=[t for t in T])
        result = [sol.y[0].tolist(), sol.y[1].tolist(), sol.y[2].tolist(), sol.y[3].tolist(), sol.y[4].tolist(), sol.y[5].tolist()]
        return result



    # A matrix
    def a(self, n, R_arm, R_DC, L_arm, L_DC, R_AC, L_AC, config):
        A = np.zeros((self.n, self.n))
        np.fill_diagonal(A, [0, -(R_arm+3*R_DC)/(L_arm+3*L_DC), -(R_arm)/(L_arm), -(R_arm)/(L_arm), -(R_arm+2*R_AC)/(L_arm+2*L_AC), -(R_arm+2*R_AC)/(L_arm+2*L_AC)])
        return A  

    # B matrix
    def b(self, n, L_arm, L_DC, L_AC, config):
        B = np.zeros((self.n, self.n))
        np.fill_diagonal(B, [0, -(1)/(L_arm+3*L_DC), -(1)/(L_arm), -(1)/(L_arm), -(1)/(L_arm+2*L_AC), -(1)/(L_arm+2*L_AC)])
        return B  


    # F matrix
    def f(self, n, L_arm, L_DC, L_AC, config):
        F = np.zeros((self.n , self.n -2))
        F[1][3] = (-(np.sqrt(6)))/(2*L_arm+6*L_DC)
        F[4][0] = (1/(L_arm+2*L_AC))
        F[4][1] = (-1/(L_arm+2*L_AC))
        F[5][0] = (np.sqrt(3)/(3*L_arm+6*L_AC))
        F[5][1] = (np.sqrt(3)/(3*L_arm+6*L_AC))
        F[5][2] = (-(np.sqrt(3))/(3*L_arm+6*L_AC))
        return F  

    def z(self, T, omega, config):
        
        for k in range(1, 4):
            self.external_noise[k-1]= self.U_AC * np.cos (omega * T - (2 * np.pi * (k-1))/3)

        self.external_noise[3] = self.U_DC
        return self.external_noise


    def vector(self, t, p, config):
       
        self.u1 = p[0] + p[1]*np.sin(2*int(np.pi)*(p[2]*t + p[3])) + p[4]*np.cos(2*int(np.pi)*(p[5]*t + p[6]))
        self.u2 = p[7] + p[8]*np.sin(2*int(np.pi)*(p[9]*t + p[10])) + p[11]*np.cos(2*int(np.pi)*(p[12]*t + p[13]))
        self.u3 = p[14] + p[15]*np.sin(2*int(np.pi)*(p[16]*t + p[17])) + p[18]*np.cos(2*int(np.pi)*(p[19]*t + p[20]))
        self.u4 = p[21] + p[22]*np.sin(2*int(np.pi)*(p[23]*t + p[24])) + p[25]*np.cos(2*int(np.pi)*(p[26]*t + p[27]))
        self.u5 = p[28] + p[29]*np.sin(2*int(np.pi)*(p[30]*t + p[31])) + p[32]*np.cos(2*int(np.pi)*(p[33]*t + p[34]))
        self.u6 = p[35] + p[36]*np.sin(2*int(np.pi)*(p[37]*t + p[38])) + p[39]*np.cos(2*int(np.pi)*(p[40]*t + p[41])) 
        self.u = np.array(self.u)
        return self.u


    # system of equations
    def MMC(self, T, x, A, B, F, config):
        dxdt = A @ x + B @ self.vector(T, self.p, config) + F @ self.z(T, self.omega, config)
        print(self.u)
        return dxdt
        
    

   
    def supports_evaluate(self):
        return True

pde_model = PDEModel()
umbridge.serve_models([pde_model], 4242) # start model server

======== Running on http://0.0.0.0:4242 ========
(Press CTRL+C to quit)
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0